<a href="https://colab.research.google.com/github/WinsalotNot/MultiClass-Perceptron---Assignment2/blob/main/Assignment_2_Multiclass_Perceptron_Andrew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTS**

In [ ]:
import numpy as np
import pandas as pd

# **DATA ORGANIZATION**

In [ ]:
# Stores data in data_file (does not include header!)
data_file = pd.read_csv('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')
print(data_file)

# Get the unqiue values in variety for classification
unique_variety = data_file['variety'].unique()
print(unique_variety)

# Take 80% of the dataset RANDOMLY as training data, random_state is the random seed used to ensure reproducibility
training_data_80 = data_file.sample(frac=0.8, random_state=45)
# Take 20% of the dataset by ELIMINATING the training data
testing_data_20 = data_file.drop(training_data_80.index)
print(f'Training Data 1: {(len(training_data_80)/len(data_file)*100)}% Testing Data 1: {(len(testing_data_20)/len(data_file)*100)}%')

# Take 70% of the dataset RANDOMLY as training data, random_state is the random seed used to ensure reproducibility
training_data_70 = data_file.sample(frac=0.7, random_state=24)
# Take 30% of the dataset by ELIMINATING the training data
testing_data_30 = data_file.drop(training_data_70.index)
print(f'Training Data 2: {(len(training_data_70)/len(data_file)*100)}% Testing Data 2: {(len(testing_data_30)/len(data_file)*100)}%')

# Take 60% of the dataset RANDOMLY as training data, random_state is the random seed used to ensure reproducibility
training_data_60 = data_file.sample(frac=0.6, random_state=23)
# Take 40% of the dataset by ELIMINATING the training data
testing_data_40 = data_file.drop(training_data_60.index)
print(f'Training Data 3: {(len(training_data_60)/len(data_file)*100)}% Testing Data 3: {(len(testing_data_40)/len(data_file)*100)}%')


     sepal.length  sepal.width  petal.length  petal.width    variety
0             5.1          3.5           1.4          0.2     Setosa
1             4.9          3.0           1.4          0.2     Setosa
2             4.7          3.2           1.3          0.2     Setosa
3             4.6          3.1           1.5          0.2     Setosa
4             5.0          3.6           1.4          0.2     Setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  Virginica
146           6.3          2.5           5.0          1.9  Virginica
147           6.5          3.0           5.2          2.0  Virginica
148           6.2          3.4           5.4          2.3  Virginica
149           5.9          3.0           5.1          1.8  Virginica

[150 rows x 5 columns]
['Setosa' 'Versicolor' 'Virginica']
Training Data 1: 80.0% Testing Data 1: 20.0%
Training Data 2: 70.0% Testing Data 2: 30.0%
Training Data 3: 60.0%

# **MULTICLASS PERCEPTRON ALGORITHM**

In [ ]:
def multiclass_perceptron(data, weight_2D_array, learning_rate, epoch):
  # Takes all row and all except last [:, :-1], then converts to numpy compatible array
  data_numpy = data.iloc[:, :-1].to_numpy()

  # Normalize input using z-score standardization, where the mean is at 0
  data_numpy = (data_numpy - np.mean(data_numpy, axis=0)) / np.std(data_numpy, axis=0)

  # Add extra synthetic features: quadratic terms
  radius_feature = np.sqrt(data_numpy[:, 0]**2 + data_numpy[:, 1]**2).reshape(-1, 1)
  x_squared = (data_numpy[:, 0] ** 2).reshape(-1, 1)
  y_squared = (data_numpy[:, 1] ** 2).reshape(-1, 1)
  xy_interaction = (data_numpy[:, 0] * data_numpy[:, 1]).reshape(-1, 1)

  # Stack new features
  data_numpy = np.hstack((data_numpy, radius_feature, x_squared, y_squared, xy_interaction))

  # Add bias to allow the boundary line to shift left or right freely (as to not be forced to go through origin [0,0])
  # Uses np.hstack to append a column of ones/zeros to the 2D array. Note: .shape[0] is the number of rows, '1' is number of column
  data_numpy = np.hstack((data_numpy, np.ones((data_numpy.shape[0], 1))))
  weight_2D_array = np.hstack((weight_2D_array, np.zeros((weight_2D_array.shape[0], 5))))

  # Each unqiue data in 'variety' are represented sequentially from 0
  result_each_indexes = data.iloc[:, -1].to_numpy()                                         # Takes all rows and ONLY the last column [:, -1], then converts to numpy compatible array
  unique_labels, results_given_int = np.unique(result_each_indexes, return_inverse=True)    # Gets an array of the labels of each uniques and an array of the unique values mapped to their respective indexes
  label_to_index = {label: idx for idx, label in enumerate(unique_labels)}                  # Shows which unqiue values are assigned to which index
  print("Label Mapping:", label_to_index)
  print("Converted Indexes:", results_given_int)

  updated_weights = weight_2D_array.copy()
  best_weights = updated_weights.copy()
  best_accuracy = 0.0

  iteration = 0
  success = False
  while (iteration < epoch):
    weighted_sums = np.dot(data_numpy, updated_weights.T) # Uses dot matrix calculation as well as tansposing the updated_weights (turning the features into rows and classes in columns)
    results_calculated = np.argmax(weighted_sums, axis=1) # Because classes are now rows, take the biggest one from each row

    correct_predictions = np.sum(results_calculated == results_given_int)
    accuracy = correct_predictions / len(data_numpy)  # Compute accuracy

    # Track best accuracy and corresponding weights
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_weights = updated_weights.copy()

    print(f'Epoch {iteration + 1}: {correct_predictions}/{len(data_numpy)} samples correctly classified.')

    if correct_predictions == len(data_numpy):  # If 100% correct, terminate early
        print(f'Converged at epoch {iteration + 1}')
        success = True
        break

    for index, (result_calculated, result_given_int) in enumerate(zip(results_calculated, results_given_int)):
      if result_calculated != result_given_int:
          updated_weights[result_given_int] += learning_rate * data_numpy[index]  # Increases the expected class
          updated_weights[result_calculated] -= learning_rate * data_numpy[index] # Decreases the predicted class

    iteration += 1

  print(f'Best accuracy: {best_accuracy * 100:.2f}%')
  print(f'Best weights:\n{best_weights}')

  if not success:
    print(f'Given {epoch} epoch, weights did not converge: {updated_weights}')

weight_2D_array = np.array([
    [0.1, 0.2, 0.3, -0.1],
    [0.5, 0.4, 0.3, -0.2],
    [0.6, 0.2, -0.4, 0.3]
])
# After Transposing:
# [
#   [0.1, 0.5, 0.6]
#   [0.2, 0.4, 0.2]
#   [0.3, 0.3, -0.4]
#   [-0.1, -0.2, 0.3]
# ]

multiclass_perceptron(training_data_80, weight_2D_array, 0.001, 10000)

Label Mapping: {'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}
Converted Indexes: [0 0 2 0 0 0 0 2 2 2 0 2 2 2 2 0 2 2 0 1 1 1 2 1 0 2 1 1 0 1 1 1 2 2 0 2 0
 0 1 0 0 1 0 1 1 0 2 1 2 0 0 1 2 0 2 0 0 1 0 1 1 2 1 2 0 0 0 0 1 0 1 2 1 1
 1 0 0 1 0 1 2 2 2 1 1 2 1 0 1 1 1 2 1 1 2 1 1 2 2 2 0 0 1 2 2 2 0 1 2 1 1
 1 0 2 2 0 0 2 0 2]
Epoch 1: 52/120 samples correctly classified.
Epoch 2: 75/120 samples correctly classified.
Epoch 3: 87/120 samples correctly classified.
Epoch 4: 81/120 samples correctly classified.
Epoch 5: 93/120 samples correctly classified.
Epoch 6: 93/120 samples correctly classified.
Epoch 7: 94/120 samples correctly classified.
Epoch 8: 95/120 samples correctly classified.
Epoch 9: 97/120 samples correctly classified.
Epoch 10: 97/120 samples correctly classified.
Epoch 11: 99/120 samples correctly classified.
Epoch 12: 97/120 samples correctly classified.
Epoch 13: 99/120 samples correctly classified.
Epoch 14: 99/120 samples correctly classified.
Epoch 15: 98/120 samples 